In [1]:
import networkx as nx
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog
from tqdm import tqdm
from itertools import combinations
from collections import defaultdict, deque
import math, random
import heapq
import scipy.sparse as sp
from collections import deque
from joblib import Parallel, delayed
from numba import njit
from typing import Tuple, Dict, List, Set, Sequence, Union
import itertools

# 1. Heuristics

In [2]:
er = nx.erdos_renyi_graph(100, 0.045, seed=42)
er.number_of_nodes(), er.number_of_edges() 

(100, 202)

In [9]:
ba = nx.barabasi_albert_graph(100, 2, seed=42)
ba.number_of_nodes(), ba.number_of_edges()

(100, 196)

In [10]:
ws = nx.watts_strogatz_graph(100, 4, 0.4, seed=42)
ws.number_of_nodes(), ws.number_of_edges()

(100, 200)

In [11]:
er = nx.gnp_random_graph(100, 0.045, seed=42)
er.number_of_nodes(), er.number_of_edges()

(100, 202)

## 1.1 Greedy + EPC (lone)

In [3]:
def sigma_exact(
    G: nx.Graph,
    S: set,
    use_tqdm: bool = False
) -> int:
    from itertools import product
    edges = list(G.edges())
    total_conn = 0.0

    for state in product([0,1], repeat=len(edges)):
        p_state = 1
        Gp = nx.Graph()
        Gp.add_nodes_from(set(G.nodes())-S)

        for (e, keep) in zip(edges, state):
            p_edge = G.edges[e]['p']
            p_state *= (p_edge if keep else (1-p_edge))

            if keep and e[0] not in S and e[1] not in S:
                Gp.add_edge(*e)

        # count connected i<j pairs in Gp−S
        for i,j in combinations(set(G.nodes())-S, 2):
            if nx.has_path(Gp, i, j):
                total_conn += p_state

    return total_conn

In [4]:
def component_sampling_epc_mc(G, S, num_samples=10_000,
                              epsilon=None, delta=None, use_tqdm=False):
  """
  Theoretic bounds: compute N = N(epsilon, delta) by the theoretical bound.
  Experimentation:  Otherwise, use the N as input for sample count.
  """

  # Surviving vertex set and its size
  V_remaining = set(G.nodes()) - S
  n_rem = len(V_remaining)

  # base case
  if n_rem < 2:
    return 0.0

  if num_samples is None:
    assert epsilon is not None and delta is not None
    P_E = sum(G.edges[u, v]['p'] for u, v in G.edges())
    coeff = 4 * (math.e - 2) * math.log(2 / delta)
    num_samples = math.ceil(coeff * n_rem * (n_rem - 1) /
                            (epsilon ** 2 * P_E))

  C2 = 0
  it = tqdm(range(num_samples), desc='Component sampling',
            total=num_samples) if use_tqdm else range(num_samples)

  for _ in it:
    u = random.choice(tuple(V_remaining))

    # BFS based on edge probabilities

    visited = {u}
    queue = [u]

    while queue:

      v = queue.pop()
      for w in G.neighbors(v):

        # flip a coin biased by the edge probability
        # w not in deleted nodes
        if w in V_remaining and random.random() < G.edges[v, w]['p']:

          # if w is not visited
          if w not in visited:
              visited.add(w)
              queue.append(w)

    # component counting
    C2 += (len(visited) - 1)

  return (n_rem * C2) / (2 * num_samples)

In [5]:
def greedy_cndp_epc(
    G: nx.Graph,
    K: int,
    num_samples: int = 10000,
    exact: bool = False,
    use_tqdm: bool = False
) -> set:
  """
  Algorithm 2 from the paper: Greedy selection of S |S| <= K
  to minimize sigma(S) via sigma_monte_carlo().

  Returns the list S (in pick order).
  """

  # S <= {Empty set} init
  S = set()

  Sigma_delta = []
  # Current sigma(S) for the empty set
  sigma_S = 0
  if exact:
    sigma_S = sigma_exact(G, S)
  else:
    sigma_S = component_sampling_epc_mc(G, S, num_samples=num_samples)

  Sigma_delta.append(sigma_S)
  # print(f"Initial sigma(S): {sigma_S}")

  if use_tqdm:
    it = tqdm(range(K), desc='Greedy selection', total=K)
  else:
    it = range(K)

  # Greedily select K nodes
  for _ in it:
    # inits
    best_j = None
    best_gain = -float('inf')
    best_sigma = None

    # find v maximizing gain sigma(S) - sigma(S ∪ j)
    for j in G:
      # Skip if j is already in S to avoid redundant calculations
      # j ∈ S
      if j in S:
        continue

      # S ∪ j = S + {j}
      if exact:
        sigma_Sj = sigma_exact(G, S | {j})
      else:
        sigma_Sj = component_sampling_epc_mc(G, S | {j}, num_samples=num_samples)

      gain = sigma_S - sigma_Sj

      # j <= argmax_{j ∈ V\S} (sigma(S) - sigma(S ∪ j))

      if gain > best_gain:
        best_gain = gain
        best_j = j
        best_sigma = sigma_Sj


    # add the best node
    if best_j is None:
      break

    S.add(best_j)
    sigma_S = best_sigma

    Sigma_delta.append(best_sigma)
    # print(f"Selected node {best_j}, gain: {best_gain}, new sigma(S): {sigma_S}")

  return S, Sigma_delta

In [27]:
t0 = time.perf_counter()

G = er.copy()
K = 10
p = 0.9

records = []
print(f"G: nodes: {G.number_of_nodes()} and edges: {G.number_of_edges()}")  

for p in tqdm(np.arange(0.0, 1.1, 0.1), desc="Processing", total=int(1.0/0.1)):
  def fresh_graph():
    H = G.copy()
    for u, v in H.edges():
      H[u][v]['p'] = p
    return H


  Sg, sigma_delta = greedy_cndp_epc(fresh_graph(), K, 10_000, False, False)
  t1 = time.perf_counter()
  t_greedy = t1 - t0

  print(f"Greedy sigma(S): S: {Sg}")
  print(f"Time: {t_greedy:.4f}s")
  print(f"Sigma Delta: {sigma_delta}")

  records.append({
    'model': 'ER',
    'p': p,
    'algo': 'Greedy',
    'time': t_greedy,
    'epc': sigma_delta[-1],
  })

G: nodes: 100 and edges: 202


Processing:  10%|█         | 1/10 [00:15<02:15, 15.08s/it]

Greedy sigma(S): S: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Time: 15.0781s
Sigma Delta: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Processing:  20%|██        | 2/10 [00:35<02:24, 18.02s/it]

Greedy sigma(S): S: {96, 65, 98, 34, 59, 75, 16, 84, 24, 27}
Time: 35.1665s
Sigma Delta: [32.35, 30.38805, 28.4249, 27.08725, 25.224, 23.08975, 21.9631, 21.00405, 20.4102, 19.4922, 17.9685]


...

Greedy sigma(S): S: {0, 64, 6, 27, 43, 12, 79, 50, 59, 94}
Time: 2706.1650s
Sigma Delta: [4532.79, 4333.6359, 4179.8911, 3988.9892, 3839.5056, 3681.4875, 3544.3123, 3380.1315, 3233.0686, 3047.08495, 2939.706]


Processing: 100%|██████████| 10/10 [57:21<00:00, 584.65s/it]

Greedy sigma(S): S: {0, 64, 6, 40, 43, 47, 50, 59, 94, 31}
Time: 3441.4722s
Sigma Delta: [4670.15, 4474.008, 4284.2366, 4115.45295, 3939.1824, 3775.29525, 3617.1952, 3453.2853, 3308.6052, 3192.33915, 3045.6495]


Processing: 11it [1:09:21, 378.29s/it]                      

Greedy sigma(S): S: {0, 64, 6, 43, 47, 81, 50, 51, 59, 94}
Time: 4161.2153s
Sigma Delta: [4746.21, 4542.14475, 4361.049, 4162.8132, 4003.2912, 3808.92525, 3641.4425, 3487.83015, 3309.4332, 3214.484, 3130.8885]


In [29]:
df = pd.DataFrame(records)
df.to_csv(f"Greedy_result_100_200_ER.csv", index=False)
df

,model,p,algo,time,epc
0,ER,0.0,Greedy,15.078099,0.0000
1,ER,0.1,Greedy,35.166469,17.9685
2,ER,0.2,Greedy,67.458208,50.3010
3,ER,0.3,Greedy,147.933775,128.0565
4,ER,0.4,Greedy,373.475165,327.8430
5,ER,0.5,Greedy,779.484757,869.1615
6,ER,0.6,Greedy,1358.731470,1880.7390
7,ER,0.7,Greedy,1990.543450,2439.3825
8,ER,0.8,Greedy,2706.164965,2939.7060
9,ER,0.9,Greedy,3441.472245,3045.6495


In [ ]:
from pulp import (
    LpProblem, LpVariable, lpSum,
    LpBinary, LpMinimize, PULP_CBC_CMD, LpStatus
)

def sample_scenarios(G_prob, T=30, rng=None):
    rng = rng or random.Random()
    scenarios = []
    for _ in range(T):
        H = nx.Graph()
        H.add_nodes_from(G_prob.nodes)
        for u, v, d in G_prob.edges(data=True):
            if rng.random() < d['p']:
                H.add_edge(u, v)
        scenarios.append(H)
    return scenarios

def solve_saa_mip_cbc(scenarios, k, msg=False):
    """
    Solve the SAA master problem with CBC and return initial set S₀ (|S₀|=k).
    """
    n = len(scenarios[0])
    T = len(scenarios)
    prob = LpProblem("k-pCND-SAA-CBC", LpMinimize)

    s = [LpVariable(f"s_{i}", cat=LpBinary) for i in range(n)]

    x = {}
    for l, H in enumerate(scenarios):
        for i, j in itertools.combinations(range(n), 2):
            x[(l, i, j)] = LpVariable(f"x_{l}_{i}_{j}", cat=LpBinary)
        for i in range(n):
            x[(l, i, i)] = 1

    prob += lpSum(s) == k

    for l, H in enumerate(scenarios):
        for i, j in itertools.combinations(range(n), 2):
            prob += x[(l, i, j)] <= 1 - s[i]
            prob += x[(l, i, j)] <= 1 - s[j]

        for i, j in itertools.combinations(range(n), 2):
            if not nx.has_path(H, i, j):
                prob += x[(l, i, j)] == 0

        for i, h, j in itertools.permutations(range(n), 3):
            ij = tuple(sorted((i, j)))
            ih = tuple(sorted((i, h)))
            hj = tuple(sorted((h, j)))
            prob += x[(l, ij[0], ij[1])] <= x[(l, ih[0], ih[1])]
            prob += x[(l, ij[0], ij[1])] <= x[(l, hj[0], hj[1])]

    prob += (1 / T) * lpSum(x.values())

    status = prob.solve(PULP_CBC_CMD(msg=msg))
    if LpStatus[status] != "Optimal":
        print("⚠ CBC ended with status:", LpStatus[status])

    S0 = {i for i in range(n) if s[i].value() > 0.5}
    return S0

def local_search(G_prob, S0, num_samples=10_000, max_iter=1000, seed=None):
    rng = random.Random(seed)
    S = set(S0)
    best_val = component_sampling_epc_mc(G_prob, S, num_samples)
    for _ in tqdm(range(max_iter), desc="Local_search", total=max_iter):
        improved = False
        for u in list(S):
            # random permutation of outside nodes
            outside = rng.sample(list(set(G_prob.nodes) - S), len(G_prob) - len(S))
            for v in outside:
                candidate = (S - {u}) | {v}      # keeps size = k
                val = component_sampling_epc_mc(G_prob, candidate, num_samples)
                if val < best_val:
                    S, best_val = candidate, val
                    improved = True
                    break
            if improved:
                break
        if not improved:
            break
    return S, best_val

def final_evaluation(G_prob, S, num_samples=100_000):
    return component_sampling_epc_mc(G_prob, S, num_samples)

def saa_algorithm(G_prob, k, T=30, ls_samples=10_000, seed=None):
    """
    Full 4-phase SAA algorithm (CBC core + numba EPC).
    Returns
    -------
    S_best      : set[int]       # final deletion set (size = k)
    epc_est     : float          # EPC estimate after local search (ls_samples draws)
    epc_final   : float          # high-accuracy EPC (100 000 draws)
    """
    scenarios = sample_scenarios(G_prob, T, rng=random.Random(seed))

    S0 = solve_saa_mip_cbc(scenarios, k, msg=False)
    S_best, epc_est = local_search(G_prob, S0, ls_samples, seed=seed)

    epc_final = final_evaluation(G_prob, S_best)

    return S_best, epc_est, epc_final

Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23
Restricted license - for non-production use only - expires 2026-11-23


: 

In [ ]:


M = 0.0443

er = nx.erdos_renyi_graph(100, 0.045, seed=42)

G = er.copy()
K = 10

records3 = []
print(f"G: nodes: {G.number_of_nodes()} and edges: {G.number_of_edges()}")  

for p in tqdm(np.arange(0.0, 1.1, 0.1), desc="Processing", total=int(1.1/0.1)):
  def fresh_graph():
    H = G.copy()
    for u, v in H.edges():
      H[u][v]['p'] = p
    return H

  t0 = time.perf_counter()
  S_best, epc_est, epc_final = saa_algorithm(fresh_graph(), k=K, T=30)
  t1 = time.perf_counter()
  t_saa = t1 - t0
  
  print(f"SAA S: S_best {S_best}")
  print(f"SAA sigma(S): {epc_final}")
  print(f"Time: {t_saa:.4f}s")

  records3.append({
    'model': 'ER',
    'p': p,
    'algo': 'SAA',
    'time': t_saa,
    'epc': epc_final,
  })

G: nodes: 100 and edges: 202


Processing:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
df3 = pd.DataFrame(records3)
df3.to_csv(f"SAA_100_200_ER.csv", index=False)

In [6]:
def greedy_epc_mis(G, k, num_samples):

  # Maximal independent set
  MIS = nx.maximal_independent_set(G)
  R = set(MIS)
  target = len(G) - k
  V = G.number_of_nodes()

  sigma_delta = []

  print(f"#MIS: {len(R)}")

  # Greedy grow R set until |R| = |V| - k
  while len(R) < target:
    best_j, best_sigma = None, float('inf')
    for j in G.nodes():
      if j in R:
        continue

      # delete node
      S_j = set(G.nodes()) - (R | {j})
      sigma = component_sampling_epc_mc(G, S=S_j, num_samples=num_samples)

      if sigma < best_sigma:
        best_sigma, best_j = sigma, j

        sigma_delta.append(best_sigma)

    R.add(best_j)
  
  D = set(G.nodes()) - R
  return D, sigma_delta

In [7]:
er = nx.erdos_renyi_graph(100, 0.045, seed=42)

G = er.copy()
K = 10

records4 = []
print(f"G: nodes: {G.number_of_nodes()} and edges: {G.number_of_edges()}")  

for p in tqdm(np.arange(0.1, 1.1, 0.1), desc="Processing", total=int(1.0/0.1)):
  def fresh_graph():
    H = G.copy()
    for u, v in H.edges():
      H[u][v]['p'] = p
    return H

  t0 = time.perf_counter()
  S_mis, sigma_delta = greedy_epc_mis(G=fresh_graph(), k=K, num_samples=100_000)
  t1 = time.perf_counter()
  t_greedy_mis = t1 - t0
  est_final_epc = component_sampling_epc_mc(fresh_graph(), S_mis, 100_000)

  print(f"Greedy S: {S_mis}")
  print(f"Greedy sigma(S): {sigma_delta}")
  print(f"final epc: {est_final_epc}")
  print(f"Time: {t_greedy_mis:.4f}s")

  records4.append({
    'model': 'ER',
    'p': p,
    'algo': 'Greedy',
    'epc': est_final_epc,
  })

G: nodes: 100 and edges: 202


Processing:   0%|          | 0/10 [00:00<?, ?it/s]

#MIS: 39


Processing:  10%|█         | 1/10 [03:08<28:13, 188.15s/it]

Greedy S: {64, 65, 34, 96, 98, 42, 12, 79, 22, 27}
Greedy sigma(S): [0.3246, 0.2038, 0.1074, 0.0994, 0.0974, 0.0924, 0.0912, 0.424145, 0.319185, 0.31857, 0.31775, 0.313035, 0.195365, 0.19229, 0.190855, 0.52311, 0.40698, 0.40299, 0.30681, 0.30177, 0.29694, 0.29442, 0.62694, 0.50396, 0.407425, 0.397965, 0.394525, 0.388935, 0.74008, 0.6039, 0.48972, 0.839475, 0.72225, 0.706275, 0.690525, 0.58815, 0.585675, 0.92368, 0.92, 0.82156, 0.7797, 0.71622, 0.70472, 0.68678, 0.67781, 1.031885, 0.903105, 0.79289, 0.78208, 1.12584, 1.02528, 1.02072, 1.01688, 1.00368, 0.86616, 1.23676, 1.12602, 1.10936, 1.101765, 0.968485, 1.36775, 1.33875, 1.27, 1.19975, 1.09225, 1.42749, 1.33824, 1.30254, 1.289535, 1.23012, 1.22451, 1.58678, 1.54674, 1.45886, 1.4378, 1.38346, 1.29636, 1.76649, 1.651215, 1.51792, 1.5158, 1.50149, 1.499105, 1.40503, 1.88757, 1.75716, 1.65888, 1.62783, 1.62459, 2.129325, 2.013825, 1.887875, 1.88375, 1.82105, 1.8139, 2.40744, 2.21816, 2.09356, 2.08544, 2.02916, 2.0202, 2.56671, 2.389725,

...

Greedy S: {0, 65, 34, 96, 42, 12, 24, 27, 93, 30}
Greedy sigma(S): [4.38086, 2.19912, 0.82764, 0.79948, 0.79002, 0.77858, 5.1138, 3.054825, 1.57365, 1.562175, 5.88547, 4.99928, 2.37774, 2.37636, 11.54978, 5.800035, 3.177905, 3.16169, 12.41808, 6.59544, 4.01088, 3.92544, 13.10848, 7.315945, 4.78779, 4.75202, 13.95975, 8.18025, 5.63325, 5.5925, 5.521, 14.747415, 9.07596, 6.45507, 6.392085, 6.38418, 15.62002, 9.83112, 7.16222, 7.15546, 7.15026, 16.32559, 10.536135, 7.98392, 17.20143, 11.36268, 10.23489, 8.79876, 18.056775, 12.152525, 10.976075, 10.96975, 20.32968, 18.40972, 13.27536, 22.572855, 20.572725, 20.425095, 16.81671, 15.55758, 15.37404, 24.69901, 22.6722, 18.89002, 17.62591, 31.629605, 29.01856, 24.803895, 21.10017, 20.006605, 33.7725, 31.5558, 26.9175, 23.3925, 23.3703, 36.877855, 30.602175, 26.74606, 26.636565, 40.60938, 30.18315, 48.118455, 46.613385, 41.69529, 38.20005, 37.060695, 36.137745, 34.35705, 52.70688, 52.45344, 50.59136, 47.53696, 42.49856, 41.8976, 40.32384, 58.613

Processing:  90%|█████████ | 9/10 [1:33:03<16:46, 1006.06s/it]

Greedy S: {0, 96, 34, 40, 42, 12, 50, 24, 58, 27}
Greedy sigma(S): [8.492535, 0.87822, 9.43005, 3.52695, 1.80012, 1.79466, 1.76547, 10.218735, 4.436955, 4.33698, 2.71674, 2.70728, 2.70341, 2.683415, 2.68019, 2.675675, 2.6746, 11.17578, 5.32444, 5.31652, 3.60492, 3.59304, 3.5904, 3.57962, 12.1464, 6.22575, 6.198075, 5.300775, 4.51575, 4.51395, 4.50045, 4.49685, 13.01133, 7.01477, 6.25646, 5.41834, 5.34037, 13.85231, 8.009035, 7.03825, 6.32949, 6.294945, 6.292125, 6.27497, 14.76384, 8.89488, 8.03736, 7.17672, 7.06032, 15.54231, 9.883055, 9.818865, 8.89644, 8.084755, 8.029875, 16.624, 12.26625, 10.73675, 9.7985, 8.98625, 8.90975, 17.290275, 13.14219, 11.457915, 10.71, 9.87309, 18.4444, 13.97656, 12.50912, 11.63604, 11.55726, 11.51462, 10.77336, 19.32062, 14.987075, 13.36236, 12.536355, 12.49316, 20.84535, 16.57449, 15.0336, 14.17824, 14.07645, 22.6259, 18.25945, 16.744475, 15.874375, 24.48516, 20.08748, 20.03568, 18.53432, 18.47608, 26.891175, 22.627005, 21.126765, 21.110235, 29.39237, 25

Processing: 100%|██████████| 10/10 [2:03:02<00:00, 738.28s/it] 

Greedy S: {64, 34, 98, 39, 50, 22, 24, 91, 93, 94}
Greedy sigma(S): [9.9568, 0.996, 0.9802, 0.9792, 10.809035, 1.990755, 1.984605, 1.97292, 1.97128, 12.06744, 3.01833, 2.99502, 2.97465, 2.96016, 13.043835, 4.01706, 3.99126, 3.9732, 3.97105, 3.96417, 14.0987, 5.01798, 4.9841, 4.97134, 4.96826, 14.92335, 5.920875, 5.912325, 16.1, 7.03018, 6.98694, 6.95037, 16.92846, 8.001985, 7.95522, 17.99064, 10.04232, 10.00248, 8.87712, 18.970105, 11.006625, 9.98375, 20.0445, 17.914, 13.0245, 12.03375, 12.02, 11.06025, 11.04075, 11.00425, 10.9265, 21.05484, 18.866685, 13.95462, 13.037385, 12.96216, 12.0411, 11.98908, 11.95644, 21.97364, 20.06264, 16.90052, 14.00776, 13.0, 23.205255, 20.99118, 18.0465, 15.057035, 14.95183, 14.00843, 23.8572, 21.96747, 19.05471, 16.07958, 15.9894, 25.8687, 21.043, 17.9311, 28.04172, 23.09664, 21.10948, 20.90144, 30.95727, 25.841235, 23.965935, 34.06021, 28.95621, 28.90778, 27.05062, 45.01877, 37.98361, 31.95912, 31.939945, 49.8279, 42.8508, 37.0953, 36.918, 55.171145, 4

In [8]:
df4 = pd.DataFrame(records4)
df4.to_csv("Greedy_MIS_100_202_ER.csv", index=False)

In [ ]:
import time
import networkx as nx
import numpy as np
from numba import njit, set_num_threads

# Restrict Numba to a single thread to avoid parallel crashes
set_num_threads(1)

def graph_to_csr(G):
    """
    Convert NetworkX G into CSR arrays and an edge-probability array.
    Returns:
      n        -- number of nodes
      row_ptr  -- int64[n+1]
      col_idx  -- int64[2|E|]
      prob_data-- float64[2|E|]
      mapping  -- dict: node -> index
    """
    mapping = {node: i for i, node in enumerate(G.nodes())}
    n = len(mapping)
    # degree counts
    deg = np.zeros(n, np.int64)
    for u, v in G.edges():
        iu, iv = mapping[u], mapping[v]
        deg[iu] += 1
        deg[iv] += 1
    # build row pointers
    row_ptr = np.empty(n+1, np.int64)
    row_ptr[0] = 0
    for i in range(n):
        row_ptr[i+1] = row_ptr[i] + deg[i]
    total = row_ptr[-1]
    # allocate CSR and prob arrays
    col_idx   = np.empty(total, np.int64)
    prob_data = np.empty(total, np.float64)
    cursor = np.zeros(n, np.int64)
    # fill
    for u, v in G.edges():
        p = G.edges[u, v]['p']
        iu, iv = mapping[u], mapping[v]
        pos = row_ptr[iu] + cursor[iu]
        col_idx[pos]   = iv
        prob_data[pos] = p
        cursor[iu]    += 1
        pos = row_ptr[iv] + cursor[iv]
        col_idx[pos]   = iu
        prob_data[pos] = p
        cursor[iv]    += 1
    return n, row_ptr, col_idx, prob_data, mapping

@njit
def epc_mc_numba(n, row_ptr, col_idx, prob_data, deletion_mask, num_samples):
    """
    Monte-Carlo EPC estimator in nopython mode (no Python objects).
    deletion_mask[i]==True => node i is deleted.
    """
    tot_C2 = 0.0
    visited = np.zeros(n, np.uint8)
    queue   = np.empty(n, np.int64)
    for si in range(num_samples):
        # pick a random surviving root
        u = np.random.randint(0, n)
        while deletion_mask[u]:
            u = np.random.randint(0, n)
        head = 0
        tail = 1
        visited[:] = 0
        visited[u] = 1
        queue[0]   = u
        comp_size  = 0
        # BFS with probability flips
        while head < tail:
            v = queue[head]
            head += 1
            comp_size += 1
            for idx in range(row_ptr[v], row_ptr[v+1]):
                w = col_idx[idx]
                if visited[w] or deletion_mask[w]:
                    continue
                if np.random.random() < prob_data[idx]:
                    visited[w]   = 1
                    queue[tail]  = w
                    tail        += 1
        tot_C2 += (comp_size - 1)
    # count survivors
    n_rem = 0
    for i in range(n):
        if not deletion_mask[i]:
            n_rem += 1
    return (n_rem * tot_C2) / (2.0 * num_samples)


def greedy_epc_mis_numba(n, row_ptr, col_idx, prob_data, mapping, MIS, k, num_samples):
    """
    MIS-seeded greedy: grow survivors from MIS up to n-k.
    Returns:
      D          -- final deletion set of size k
      sigma_delta-- EPC history (increasing as survivors grow)
    """
    # build deletion mask: True=>deleted
    deletion_mask = np.ones(n, np.bool_)
    for node in MIS:
        deletion_mask[mapping[node]] = False
    survivors = set(MIS)
    target    = n - k
    sigma_delta = []
    # initial EPC
    curr_sigma = epc_mc_numba(n, row_ptr, col_idx, prob_data, deletion_mask, num_samples)
    sigma_delta.append(curr_sigma)
    # greedy grow survivors
    while len(survivors) < target:
        best_val  = np.inf
        best_node = None
        for node, idx in mapping.items():
            if node in survivors:
                continue
            # try keeping `node`
            deletion_mask[idx] = False
            val = epc_mc_numba(n, row_ptr, col_idx, prob_data, deletion_mask, num_samples)
            deletion_mask[idx] = True
            if val < best_val:
                best_val, best_node = val, node
        # commit best keep
        deletion_mask[mapping[best_node]] = False
        survivors.add(best_node)
        curr_sigma = best_val
        sigma_delta.append(curr_sigma)
    # final deletions = everything not in survivors
    D = set(mapping.keys()) - survivors
    return D, sigma_delta

In [23]:
t0 = time.perf_counter()

er = nx.erdos_renyi_graph(100, 0.045, seed=42)

G = er.copy()
K = 10

records4 = []
print(f"G: nodes: {G.number_of_nodes()} and edges: {G.number_of_edges()}")  

for p in tqdm(np.arange(0.1, 1.1, 0.1), desc="Processing", total=int(1.0/0.1)):
  def fresh_graph():
    H = G.copy()
    for u, v in H.edges():
      H[u][v]['p'] = p
    return H


  S_mis, sigma_delta = greedy_epc_mis_numba(G=fresh_graph(), k=K, num_samples=100_000)
  t1 = time.perf_counter()
  t_greedy_mis = t1 - t0

  print(f"Greedy S: {S_mis}")
  print(f"Greedy sigma(S): {sigma_delta}")
  print(f"Time: {t_greedy_mis:.4f}s")

  records4.append({
    'model': 'ER',
    'p': p,
    'algo': 'Greedy',
    'time': t_greedy_mis,
    'epc': sigma_delta[-1],
  })

G: nodes: 100 and edges: 202


Processing:  10%|█         | 1/10 [00:09<01:23,  9.33s/it]

Greedy S: {96, 34, 39, 10, 42, 12, 15, 52, 24, 27}
Greedy sigma(S): [0.0, 0.094095, 0.19257, 0.29412, 0.38412, 0.493875, 0.58006, 0.694895, 0.79392, 0.879795, 0.99475, 1.08936, 1.2181, 1.380385, 1.53387, 1.75615, 1.95636, 2.18082, 2.41425, 2.60898, 2.8653, 3.07745, 3.29747, 3.52674, 3.73984, 4.01635, 4.23852, 4.500725, 4.86064, 5.23572, 5.544, 6.026125, 6.42384, 6.831705, 7.3815, 7.888875, 8.40522, 8.87656, 9.53628, 10.091855, 10.6956, 11.29707, 11.97774, 12.757515, 13.32114, 13.974, 14.79587, 15.88881, 16.5792, 17.517425, 18.51975]
Time: 9.3323s


Processing:  20%|██        | 2/10 [00:19<01:16,  9.61s/it]

Greedy S: {64, 65, 34, 96, 98, 10, 12, 24, 90, 27}
Greedy sigma(S): [0.0, 0.1952, 0.395445, 0.60627, 0.787115, 1.01574, 1.2033, 1.41542, 1.590715, 1.82136, 2.04673, 2.2585, 2.45361, 2.65642, 2.916325, 3.13578, 3.479025, 3.84468, 4.301505, 4.75832, 5.19318, 5.6667, 6.288795, 6.69135, 7.301385, 7.6864, 8.4292, 8.92716, 9.678485, 10.438, 11.47401, 12.52685, 13.40338, 14.49468, 15.33511, 16.64445, 17.68425, 18.8784, 20.295275, 21.83337, 23.984795, 25.518, 27.65826, 29.99642, 32.41897, 34.81086, 38.6801, 42.22428, 46.786425, 51.67404, 58.28966, 66.12435]
Time: 19.1319s


Processing:  30%|███       | 3/10 [00:31<01:17, 11.07s/it]

Greedy S: {96, 65, 34, 5, 10, 12, 52, 54, 90, 27}
Greedy sigma(S): [0.0, 0.300105, 0.5914, 0.916965, 1.2201, 1.52478, 1.80422, 2.15955, 2.46468, 2.79133, 3.08064, 3.379285, 3.71, 4.02798, 4.33602, 4.848705, 5.46021, 6.2601, 6.9006, 7.78848, 8.53818, 9.413155, 10.431, 11.232235, 12.16068, 13.06116, 13.80992, 14.98315, 16.25316, 17.38382, 18.1764, 20.155245, 21.6601, 23.81553, 26.50716, 28.509055, 31.25279, 34.878375, 37.5763, 43.965075, 51.22572, 57.92122, 65.99, 77.580585, 88.56615, 108.083845, 126.51072, 142.29085, 171.47712, 206.420985, 249.62696, 291.998765, 342.33615]
Time: 31.9363s


: 